In [17]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias


df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)

# Randoms solo para asegurarse que los features esten bien hechos
# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_train = utils.dolarizar_df(df_train)
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.2)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)


# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_test = utils.pesificar_df(df_test)
# df_train = df_train.sample(frac=1).reset_index(drop=True)


In [18]:
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

# df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

## Búsqueda hiperparámetros

In [4]:
# features = ['habitaciones', 
#             'garages', 
#             'banos',
#             'antiguedad',
#             'metroscubiertos', 
#             'metrostotales', 
#             'lat', 'lng',
#             'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

# features_test = ['prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
#                  'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
#                  'count_id_zona', 'count_ciudad', 'puntaje', 
#                      'count_tipo_propiedad_ciudad', 
#                  'promedio_precio_tipo_propiedad_ciudad_gen',
#                  'count_id_zona'
#                  'dias_desde_datos',
#                  'meses_desde_datos',
#                  'porcentaje_metros',
#                  'promedio_precio_hbg_tipo_propiedad']

# features += features_test

# features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe

features =['antiguedad', 'metroscubiertos', 'metrostotales', 'lat', 'lng',
       'porcentaje_metros', 'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_ciudad', 'varianza_precio_ciudad', 'promedio_id_zona',
       'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen',
       'count_tipo_propiedad_ciudad', 'varianza_por_mes',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_hbg_tipo_propiedad',
       'promedio_precio_hbg_tipo_propiedad_provincia', 'puntaje', 'idf_titulo',
       'idf_descripcion', 'distancia_ciudad_centrica']

def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth, test_size = args
    
    df_train_h = df_train_f.sample(frac=1).reset_index(drop=True)
    
    x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_h, 'precio', features, test_size=test_size)

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=150,
                    early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)
    return utils.MAE(df_test_f['precio'].values, y_pred_test)

space = [hp.quniform('num_leaves', 30, 180, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 30, 1),
        hp.uniform('test_size', 0.05, 0.25)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=200, verbose=1)

display(hps)

Training until validation scores don't improve for 15 rounds
Early stopping, best iteration is:                   
[8]	valid_0's l1: 611386
Training until validation scores don't improve for 15 rounds                 
Did not meet early stopping. Best iteration is:                              
[149]	valid_0's l1: 492357
Training until validation scores don't improve for 15 rounds                 
Early stopping, best iteration is:                                           
[28]	valid_0's l1: 566682
Training until validation scores don't improve for 15 rounds                 
Early stopping, best iteration is:                                           
[13]	valid_0's l1: 720840
Training until validation scores don't improve for 15 rounds                 
Early stopping, best iteration is:                                           
[98]	valid_0's l1: 550317
Training until validation scores don't improve for 15 rounds                 
Early stopping, best iteration is:                   

Early stopping, best iteration is:                                            
[15]	valid_0's l1: 547649
Training until validation scores don't improve for 15 rounds                  
Early stopping, best iteration is:                                            
[10]	valid_0's l1: 563031
Training until validation scores don't improve for 15 rounds                  
Early stopping, best iteration is:                                            
[6]	valid_0's l1: 592310
Training until validation scores don't improve for 15 rounds                  
Early stopping, best iteration is:                                            
[28]	valid_0's l1: 526893
Training until validation scores don't improve for 15 rounds                  
Early stopping, best iteration is:                                            
[15]	valid_0's l1: 547584
Training until validation scores don't improve for 15 rounds                  
Early stopping, best iteration is:                                            
[2

Training until validation scores don't improve for 15 rounds                   
Did not meet early stopping. Best iteration is:                                
[150]	valid_0's l1: 531806
Training until validation scores don't improve for 15 rounds                   
Early stopping, best iteration is:                                             
[128]	valid_0's l1: 514615
Training until validation scores don't improve for 15 rounds                   
Did not meet early stopping. Best iteration is:                                
[148]	valid_0's l1: 532197
Training until validation scores don't improve for 15 rounds                   
Did not meet early stopping. Best iteration is:                                
[150]	valid_0's l1: 492566
Training until validation scores don't improve for 15 rounds                   
Early stopping, best iteration is:                                             
[35]	valid_0's l1: 571753
Training until validation scores don't improve for 15 rounds      

{'bagging_fraction': 0.7851550830043029,
 'bagging_freq': 78.0,
 'feature_fraction': 0.30695183626263145,
 'learning_rate': 0.0955953803928466,
 'max_depth': 15.0,
 'num_leaves': 154.0,
 'test_size': 0.10997125282541062}

## Evaluación features

In [ ]:
#features = ['antiguedad', 'habitaciones', 'garages', 'banos',
#       'metroscubiertos', 'metrostotales', 
#            'lat', 'lng',
#       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

#features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
#                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
#                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
#                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia',
                 'varianza_id_zona', 'promedio_id_zona_log', 'tam_ambientes', 'metros_cubiertos_normalizados', 
                 'dias_desde_datos', 'meses_desde_datos']

features += features_test

features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe

features_remove = [['lat', 'lng'], ['antiguedad'], ['habitaciones'], ['garages'], ['banos'], ['metroscubiertos'], ['metrostotales'],
                   ['prop_frecuente'], ['top_provincia'], ['porcentaje_metros'], ['diferencia_metros'], 
                   ['promedio_precio_ciudad'],  ['promedio_por_mes'], ['anio'], ['promedio_id_zona'], ['promedio_id_zona_log'],
                   ['promedio_id_zona', 'promedio_id_zona_log'], ['promedio_precio_tipo_propiedad'],  
                   ['promedio_precio_hbg_tipo_propiedad'], ['count_id_zona'], ['count_ciudad'], 
                   ['puntaje'],  ['count_tipo_propiedad'], ['count_tipo_propiedad_ciudad'],  
                   ['promedio_precio_tipo_propiedad_ciudad_gen'], ['promedio_precio_hbg_tipo_propiedad_provincia'], 
                   ['varianza_id_zona'], ['tam_ambientes'], ['metros_cubiertos_normalizados'], ['dias_desde_datos'], 
                   ['meses_desde_datos']]

hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}

bagging_fraction = hps['bagging_fraction']
bagging_freq = int(hps['bagging_freq'])
feature_fraction = hps['feature_fraction']
learning_rate = hps['learning_rate']
num_leaves = int(hps['num_leaves'])
max_depth = int(hps['max_depth'])
test_size = hps['test_size']

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,
    'feature_fraction': feature_fraction,
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'max_depth': max_depth,
    'verbose': 0
}


base_train = 0
base_test = 0
base_eval = 0
for i in [['None']] + features_remove:
    
    features_new = [f for f in features if f not in i]
    if len(features_new) == len(features) and i != ['None']:
        print(f'{i} no encontrado')
        conitnue
    
    x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features_new, test_size=test_size)

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=1000,
                    early_stopping_rounds=15,
                    verbose_eval=-1)

    y_pred_test = gbm.predict(x_test, raw_score=True, num_iteration=gbm.best_iteration)
    y_pred_train = gbm.predict(x_train, raw_score=True, num_iteration=gbm.best_iteration)
    y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features_new), num_iteration=gbm.best_iteration)

    gbm_mae_train = utils.MAE(y_train, y_pred_train)
    gbm_mae = utils.MAE(y_test, y_pred_test)
    gbm_mae_eval = utils.MAE(df_test_f['precio'].values, y_pred_eval)

    print(f"MAE LightGBM (train): {gbm_mae_train:.5f}")
    print(f"MAE LightGBM (test): {gbm_mae:.5f}")
    print(f"MAE LightGBM (eval): {gbm_mae_eval:.5f}")
    if i != ['None']:
        print(f"Overfitting (base_eval - base_test) - (eval - test) - {i}: {(base_eval - base_test) - (gbm_mae_eval - gbm_mae)}")
        print(f"Diff evaluation (base_eval - eval)                  - {i}: {base_eval - gbm_mae_eval}")
        print(f"Diff train (base_train - train)                     - {i}: {base_train - gbm_mae_train}")
    else:
        base_train = gbm_mae_train
        base_test = gbm_mae
        base_eval = gbm_mae_eval

## Evaluación modelo final

In [24]:
features = ['habitaciones', 
            'garages', 
            'banos',
            'antiguedad',
           'metroscubiertos', 
            'metrostotales',
            'lat_norm', 'lng_norm'
            'gimnasio', 'usosmultiples', 'piscina']

features_test = ['prop_frecuente', 'top_provincia', 'promedio_precio_ciudad', 
                 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'count_id_zona', 'count_ciudad', 'puntaje', 
                     'count_tipo_propiedad_ciudad', 
                 'promedio_precio_tipo_propiedad_ciudad_gen',
                 'count_id_zona'
                 'dias_desde_datos',
                 'meses_desde_datos',
                 'porcentaje_metros',
                 'distancia_ciudad_centrica', 'puntaje']

features += features_test

# features += features_test

# features += cols_tipodepropiedad_ohe + cols_provincia_ohe + cols_zona_ohe


# features = ['antiguedad', 'metroscubiertos', 'metrostotales', 'lat', 'lng',
#        'porcentaje_metros', 'promedio_metros_tipo_propiedad',
#        'dias_desde_datos', 'tam_ambientes', 'promedio_precio_ciudad',
#        'varianza_precio_ciudad', 'promedio_id_zona', 'varianza_id_zona',
#        'count_id_zona', 'promedio_precio_tipo_propiedad_ciudad',
#        'count_tipo_propiedad_ciudad',
#        'promedio_precio_habitaciones_banos_garages',
#        'promedio_precio_hbg_tipo_propiedad',
#        'promedio_precio_hbg_tipo_propiedad_provincia', 'puntaje']
    
    
# hps = {'bagging_fraction': 0.8988911725316586,
#  'bagging_freq': 22.0,
#  'feature_fraction': 0.6622442122619671,
#  'learning_rate': 0.16422725363286422,
#  'max_depth': 22.0,
#  'num_leaves': 180.0,
#  'test_size': 0.20892455926004772}

#hps = {'bagging_fraction': 0.8999882607358867,
# 'bagging_freq': 95.0,
# 'feature_fraction': 0.2570109385381975,
# 'learning_rate': 0.13601832720254403,
# 'max_depth': 26.0,
# 'num_leaves': 175.0,
# 'test_size': 0.08363501292068126}

hps ={'bagging_fraction': 0.7851550830043029,
 'bagging_freq': 78.0,
 'feature_fraction': 0.30695183626263145,
 'learning_rate': 0.0955953803928466,
 'max_depth': 15.0,
 'num_leaves': 154.0,
 'test_size': 0.10997125282541062}


bagging_fraction = hps['bagging_fraction']
bagging_freq = int(hps['bagging_freq'])
feature_fraction = hps['feature_fraction']
learning_rate = hps['learning_rate']
num_leaves = int(hps['num_leaves'])
max_depth = int(hps['max_depth'])
test_size = hps['test_size']

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,
    'feature_fraction': feature_fraction,
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'max_depth': max_depth,
    'verbose': 0
}

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=test_size)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1000,
                early_stopping_rounds=25,
                verbose_eval=1)

y_pred_test = gbm.predict(x_test, num_iteration=gbm.best_iteration)
y_pred_train = gbm.predict(x_train, num_iteration=gbm.best_iteration)

y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)

df_test_f['target'] = y_pred_eval
# df_test_f = utils.pesificar_df(df_test_f, 'target', 'target')

gbm_mae_train = utils.MAE(y_train, y_pred_train)
gbm_mae = utils.MAE(y_test, y_pred_test)
gbm_mae_eval = utils.MAE(df_test_f['precio'].values, df_test_f['target'].values)

print(f"MAE LightGBM (train): {gbm_mae_train:.5f}")
print(f"MAE LightGBM (test): {gbm_mae:.5f}")
print(f"MAE LightGBM (eval): {gbm_mae_eval:.5f}")

[1]	valid_0's l1: 1.52926e+06
Training until validation scores don't improve for 25 rounds
[2]	valid_0's l1: 1.42517e+06
[3]	valid_0's l1: 1.34755e+06
[4]	valid_0's l1: 1.27262e+06
[5]	valid_0's l1: 1.21591e+06
[6]	valid_0's l1: 1.14683e+06
[7]	valid_0's l1: 1.08677e+06
[8]	valid_0's l1: 1.02308e+06
[9]	valid_0's l1: 971788
[10]	valid_0's l1: 919941
[11]	valid_0's l1: 876932
[12]	valid_0's l1: 842422
[13]	valid_0's l1: 820376
[14]	valid_0's l1: 789815
[15]	valid_0's l1: 759169
[16]	valid_0's l1: 738829
[17]	valid_0's l1: 718223
[18]	valid_0's l1: 706449
[19]	valid_0's l1: 687248
[20]	valid_0's l1: 673587
[21]	valid_0's l1: 658858
[22]	valid_0's l1: 646800
[23]	valid_0's l1: 638669
[24]	valid_0's l1: 629686
[25]	valid_0's l1: 620753
[26]	valid_0's l1: 611910
[27]	valid_0's l1: 605205
[28]	valid_0's l1: 600783
[29]	valid_0's l1: 595855
[30]	valid_0's l1: 590971
[31]	valid_0's l1: 587726
[32]	valid_0's l1: 583193
[33]	valid_0's l1: 579627
[34]	valid_0's l1: 576564
[35]	valid_0's l1: 57187

[307]	valid_0's l1: 493168
[308]	valid_0's l1: 493143
[309]	valid_0's l1: 493173
[310]	valid_0's l1: 493165
[311]	valid_0's l1: 493163
[312]	valid_0's l1: 493182
[313]	valid_0's l1: 493081
[314]	valid_0's l1: 493028
[315]	valid_0's l1: 493050
[316]	valid_0's l1: 493023
[317]	valid_0's l1: 492991
[318]	valid_0's l1: 492831
[319]	valid_0's l1: 492762
[320]	valid_0's l1: 492622
[321]	valid_0's l1: 492563
[322]	valid_0's l1: 492514
[323]	valid_0's l1: 492521
[324]	valid_0's l1: 492504
[325]	valid_0's l1: 492366
[326]	valid_0's l1: 492310
[327]	valid_0's l1: 492282
[328]	valid_0's l1: 492264
[329]	valid_0's l1: 492166
[330]	valid_0's l1: 492162
[331]	valid_0's l1: 492199
[332]	valid_0's l1: 492047
[333]	valid_0's l1: 492024
[334]	valid_0's l1: 492029
[335]	valid_0's l1: 491996
[336]	valid_0's l1: 491974
[337]	valid_0's l1: 491967
[338]	valid_0's l1: 491915
[339]	valid_0's l1: 491873
[340]	valid_0's l1: 491781
[341]	valid_0's l1: 491815
[342]	valid_0's l1: 491826
[343]	valid_0's l1: 491812
[

In [ ]:
import shap 
# shap.initjs()

df_test_shap = utils.filtrar_features(df_test_f, features)

explainer = shap.TreeExplainer(gbm)
shap_values = explainer.shap_values(df_test_shap)

In [ ]:
# shap.force_plot(explainer.expected_value, shap_values, df_test_shap)

In [ ]:
shap.summary_plot(shap_values, df_test_shap)

## Evaluación df_test completo

In [ ]:
df_train = pd.read_csv('./data/train.csv',)

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_test = features.llenar_nulls(df_test, hgb_mean=True)
df_train = features.llenar_nulls(df_train, hgb_mean=True)

df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=250, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=250, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', N=250, df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=250, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=250, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', N=250, df_aux=df_test_f)

df_train_f = f_textos.features_textos(df_train_f)
df_test_f = f_textos.features_textos(df_test_f)

# df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
# df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)


features =['habitaciones', 'garages', 'banos','metroscubiertos', 'metrostotales', 'lat', 'lng', 'metrostotales_log',
 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros', 'metroscubiertos_bins_perc', 'promedio_metros_tipo_propiedad',
 'promedio_metros_cub_tipo_propiedad', 'top_provincia', 'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia',
 'anio', 'mes', 'dia', 'trimestre', 'dias_desde_datos', 'meses_desde_datos', 'antiguedad_bins_unif',
 'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes', 'metros_totales_normalizados',
 'metros_cubiertos_normalizados', 'promedio_precio_provincia', 'promedio_precio_ciudad',
 'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
 'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona', 'promedio_precio_tipo_propiedad',
 'promedio_precio_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen',
 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
 'promedio_precio_habitaciones', 'promedio_precio_habitaciones_banos_garages', 'promedio_precio_banos_garages',
 'promedio_precio_hbg_tipo_propiedad', 'promedio_precio_hbg_tipo_propiedad_provincia',
 'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'idf_titulo', 'idf_descripcion',
 'peso_descripcion']

# hps = {'bagging_fraction': 0.806451877022587,
#  'bagging_freq': 62.0,
#  'feature_fraction': 0.5379925983440028,
#  'learning_rate': 0.1363027714646826,
#  'max_depth': 11.0,
#  'num_leaves': 113.0,
#  'test_size': 0.09575190901892519}


hps = {'bagging_fraction': 0.8988911725316586,
 'bagging_freq': 22.0,
 'feature_fraction': 0.6622442122619671,
 'learning_rate': 0.16422725363286422,
 'max_depth': 22.0,
 'num_leaves': 180.0,
 'test_size': 0.13892455926004772}


bagging_fraction = hps['bagging_fraction']
bagging_freq = int(hps['bagging_freq'])
feature_fraction = hps['feature_fraction']
learning_rate = hps['learning_rate']
num_leaves = int(hps['num_leaves'])
max_depth = int(hps['max_depth'])
test_size = hps['test_size']

params = {
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,
    'feature_fraction': feature_fraction,
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'max_depth': max_depth,
    'verbose': 0
}

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=test_size)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)


gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1000,
                early_stopping_rounds=15,
                verbose_eval=1)

y_pred_test = gbm.predict(x_test, raw_score=True, num_iteration=gbm.best_iteration)
y_pred_train = gbm.predict(x_train, raw_score=True, num_iteration=gbm.best_iteration)
y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)

gbm_mae_train = utils.MAE(y_train, y_pred_train)
gbm_mae = utils.MAE(y_test, y_pred_test)

print(f"MAE LightGBM (train): {gbm_mae_train:.5f}")
print(f"MAE LightGBM (test): {gbm_mae:.5f}")

In [ ]:
y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)
df_test_f['target'] = y_pred_eval

# df_test_f = utils.pesificar_df(df_test_f, 'target', 'target')
df_test_f[['id', 'target']].to_csv('respuesta25.csv', index=False)

In [ ]:
utils.filtrar_features(df_test_f, features).columns.equals(x_train.columns)

In [ ]:
df_test_f.shape